In [1]:
import pandas as pd
import numpy as np
import os
from arcgis.gis import *
gis = GIS()

strCenters = "WC2050Centers"
strTAZ     = "TAZ"
strCities  = "Cities"
strTAZID   = "CO_TAZID"
strAtoUstm = "ATO19_USTM.csv"
strAtoWfrc = "ATO19_WFRC.csv"

dirRoot         = r"E:\GitHub\Housing-Suitability-Weber-County"
dirData         = os.path.join(dirRoot,'data'        )
dirIntermediate = os.path.join(dirRoot,'intermediate')

strTAZWhere     = "TAZ.CO_FIPS = 57"
strTAZwithATO   = "TAZwithATO.csv"

# get project and map
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Map")[0]

In [2]:
for lyr in m.listLayers(strCenters):
    lyrCenters = lyr
    
lyrCenters.definitionQuery = "AreaType IN ('Metropolitan Center', 'Neighborhood Center', 'Urban Center', 'City Center')"

sym = lyrCenters.symbology

sym.updateRenderer('UniqueValueRenderer')
sym.renderer.fields = ['AreaType']

sym.renderer.addValues({"Neighborhood Center" : ["Neighborhood Center"]})
sym.renderer.addValues({"City Center"         : ["City Center"        ]})
sym.renderer.addValues({"Urban Center"        : ["Urban Center"       ]})
sym.renderer.addValues({"Metropolitan Center" : ["Metropolitan Center"]})

lyrCenters.symbology = sym
lyrCenters.transparency = 30

arcpy.management.CopyFeatures(strCenters, "Centers")

# delete unneeded fields
fldlist = arcpy.ListFields("Centers")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID_1','AreaType','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("Centers",[fld.name])

print("DONE!")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset WC2050Centers does not exist or is not supported
WARNING 000725: Output Feature Class: Dataset E:\GitHub\Housing-Suitability-Weber-County\gis\HousingAffordability.gdb\Centers already exists.
Failed to execute (CopyFeatures).


In [3]:
# Calculate ATO Scores by City 
lyrTAZ = m.listLayers(strTAZ)[0]

lyrTAZ.definitionQuery = strTAZWhere

arcpy.management.RemoveJoin(strTAZ)
arcpy.management.AddJoin(strTAZ, strTAZID               , strAtoUstm, strTAZID)
arcpy.management.AddJoin(strTAZ, strTAZ + "." + strTAZID, strAtoWfrc, strTAZID)

arcpy.conversion.TableToTable(strTAZ, "E:\\GitHub\\Housing-Affordability-Weber-County\\intermediate", strTAZwithATO, strTAZWhere)

arcpy.management.RemoveJoin(strTAZ)

## delete unneeded fields
#fldlist = arcpy.ListFields(strTAZwithATO)
#
#for fld in fldlist: 
#    if (fld.name not in ['CO_TAZID','AUTO_JB','AUTO_JB_1','TRAN_JB']):
#        print("deleting field: " + fld.name)
#        arcpy.DeleteField_management(strTAZwithATO,[fld.name])
#
#print("DONE!")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Layer Name or Table View: Dataset TAZ does not exist or is not supported
Failed to execute (RemoveJoin).


In [46]:
## Create cities
#arcpy.analysis.PairwiseDissolve(strTAZ, strCities, "CITY_NAME")
#
#for lyr in m.listLayers(strCities):
#    lyrCities = lyr
#    
#sym = lyrCenters.symbology
#
##sym.updateRenderer('UniqueValueRenderer')
##sym.renderer.fields = ['CITY_NAME']
#
##sym.renderer.addValues({"Neighborhood Center" : ["Neighborhood Center"]})
##sym.renderer.addValues({"City Center"         : ["City Center"        ]})
##sym.renderer.addValues({"Urban Center"        : ["Urban Center"       ]})
##sym.renderer.addValues({"Metropolitan Center" : ["Metropolitan Center"]})


In [4]:
# read csv into pandas dataframe
dfATO = pd.read_csv(os.path.join(dirIntermediate,strTAZwithATO))
dfATO

,OID_,TAZID,SA_TAZID,CO_TAZID,SUBAREAID,EXTERNAL,ACRES,DEVACRES,X,Y,ADJ_XY,CO_FIPS,CO_NAME,DISTSUPER,DSUP_NAME,DISTLRG,DLRG_NAME,DISTMED,DMED_NAME,DISTSML,DSML_NAME,CITY_NAME,IMX_GENRL,IMX_COALIX,IMX_OLGAIX,IMX_OLGAXI,IMX_PETRIX,TAZID_1,CO_TAZID_1,DEVACRES_1,HH,JOB,AUTO_JB,AUTO_HH,COMP_AUTO,AUTO_JB_WT,AUTO_HH_WT,TAZID_12,CO_TAZID_12,DEVACRES_12,HH_1,JOB_1,AUTO_JB_1,AUTO_HH_1,TRAN_JB,TRAN_HH,COMP_AUTO_1,COMP_TRAN,AUTO_JB_WT_1,AUTO_HH_WT_1,TRAN_JB_WT,TRAN_HH_WT
0,6145,3709,144,570144,1,0,385.7500,385.750,410560.675910,4.575721e+06,0,57,WEBER,14,Wastch Front Shoulder West/North,2,North and West Weber,6,NW Weber,24,NW Weber,Plain City,0.0,0.0,0.0,0.0,0.0,3709,570144,385.750,31.2,0.0,77215,54865,77215,2409122,0.0,144.0,570144.0,385.75,31.2,0.0,74352.0,52754.0,0.0,0.0,74352.0,0.0,2319780.0,0.0,0.0,0.0
1,6146,3708,143,570143,1,0,2171.7200,2026.240,408302.688400,4.575422e+06,0,57,WEBER,14,Wastch Front Shoulder West/North,2,North and West Weber,6,NW Weber,23,NW Weber,Weber County,0.0,0.0,0.0,0.0,0.0,3708,570143,2026.240,50.3,0.0,64942,47832,64942,3266569,0.0,143.0,570143.0,2026.24,50.3,0.0,62556.0,46107.0,0.0,0.0,62556.0,0.0,3146589.0,0.0,0.0,0.0
2,6147,3711,146,570146,1,0,376.7200,376.720,410207.499380,4.573881e+06,0,57,WEBER,14,Wastch Front Shoulder West/North,2,North and West Weber,6,NW Weber,24,NW Weber,Plain City,0.0,0.0,0.0,0.0,0.0,3711,570146,376.720,74.5,2.0,80559,58486,80211,6001666,116972.0,146.0,570146.0,376.72,74.5,2.0,78115.0,56697.0,0.0,0.0,77801.0,0.0,5819536.0,113393.0,0.0,0.0
3,6148,3714,149,570149,1,0,602.0500,602.050,411636.611210,4.572499e+06,0,57,WEBER,14,Wastch Front Shoulder West/North,2,North and West Weber,6,NW Weber,24,NW Weber,Weber County,0.0,0.0,0.0,0.0,0.0,3714,570149,602.050,341.3,39.7,97162,68016,95270,33161221,2700246.0,149.0,570149.0,602.05,341.3,39.7,94068.0,65810.0,0.0,0.0,92361.0,0.0,32105457.0,2612648.0,0.0,0.0
4,6150,3715,150,570150,1,0,681.8100,675.270,412909.138100,4.575736e+06,0,57,WEBER,14,Wastch Front Shoulder West/North,2,North and West Weber,6,NW Weber,25,NW Weber,Farr West,0.0,0.0,0.0,0.0,0.0,3715,570150,675.270,535.2,393.7,91894,63838,83338,49181906,25132912.0,150.0,570150.0,675.27,535.2,393.7,88455.0,61292.0,0.0,0.0,80602.0,0.0,47341147.0,24130726.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,4755,3552,0,57079,0,0,16388.7199,12955.315,444908.277480,4.581932e+06,0,57,WEBER,1,Northeast Utah,1,NaN,2,NaN,2,NaN,Balance of Ogden Valley,0.0,0.0,0.0,0.0,0.0,3552,57079,12955.315,5.3,0.0,2189,1696,2189,11603,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,4756,3553,0,57080,0,0,7674.0731,5613.934,447503.210853,4.576659e+06,0,57,WEBER,1,Northeast Utah,1,NaN,2,NaN,2,NaN,Balance of Ogden Valley,0.0,0.0,0.0,0.0,0.0,3553,57080,5613.934,48.6,0.0,6160,4724,6160,299376,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,4757,3555,0,57082,0,0,8105.2724,5060.994,450785.505024,4.568315e+06,0,57,WEBER,1,Northeast Utah,1,NaN,2,NaN,2,NaN,Balance of Ogden Valley,0.0,0.0,0.0,0.0,0.0,3555,57082,5060.994,0.0,0.0,3773,2915,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,4758,3556,0,57083,0,0,23365.8890,13417.661,458520.316196,4.571356e+06,0,57,WEBER,1,Northeast Utah,1,NaN,2,NaN,2,NaN,Balance of Ogden Valley,0.0,0.0,0.0,0.0,0.0,3556,57083,13417.661,0.0,0.0,1996,1557,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# manually change cities for a given TAZ ID
dTAZCityEdits = [
    ['Ogden Valley'    ,  57001],
    ['Ogden Valley'    ,  57002],
    ['Ogden Valley'    ,  57003],
    ['Ogden Valley'    ,  57004],
    ['Ogden Valley'    ,  57005],
    ['Ogden Valley'    ,  57006],
    ['Ogden Valley'    ,  57007],
    ['Ogden Valley'    ,  57008],
    ['Ogden Valley'    ,  57009],
    ['Ogden Valley'    ,  57010],
    ['Ogden Valley'    ,  57011],
    ['Ogden Valley'    ,  57012],
    ['Ogden Valley'    ,  57013],
    ['Liberty'         ,  57014],
    ['Liberty'         ,  57015],
    ['Liberty'         ,  57016],
    ['Ogden Valley'    ,  57017],
    ['Liberty'         ,  57018],
    ['Liberty'         ,  57019],
    ['Liberty'         ,  57020],
    ['Liberty'         ,  57021],
    ['Liberty'         ,  57022],
    ['Liberty'         ,  57023],
    ['Ogden Valley'    ,  57024],
    ['Ogden Valley'    ,  57025],
    ['Ogden Valley'    ,  57026],
    ['Liberty'         ,  57027],
    ['Liberty'         ,  57028],
    ['Wolf Creek'      ,  57029],
    ['Wolf Creek'      ,  57030],
    ['Wolf Creek'      ,  57031],
    ['Ogden Valley'    ,  57032],
    ['Eden'            ,  57033],
    ['Eden'            ,  57034],
    ['Eden'            ,  57035],
    ['Ogden Valley'    ,  57036],
    ['Ogden Valley'    ,  57037],
    ['Ogden Valley'    ,  57038],
    ['Ogden Valley'    ,  57039],
    ['Ogden Valley'    ,  57040],
    ['Ogden Valley'    ,  57041],
    ['Ogden Valley'    ,  57042],
    ['Ogden Valley'    ,  57043],
    ['Ogden Valley'    ,  57044],
    ['Ogden Valley'    ,  57045],
    ['Ogden Valley'    ,  57046],
    ['Ogden Valley'    ,  57047],
    ['Ogden Valley'    ,  57048],
    ['Ogden Valley'    ,  57049],
    ['Ogden Valley'    ,  57050],
    ['Ogden Valley'    ,  57051],
    ['Ogden Valley'    ,  57052],
    ['Ogden Valley'    ,  57053],
    ['Ogden Valley'    ,  57054],
    ['Ogden Valley'    ,  57055],
    ['Ogden Valley'    ,  57056],
    ['Ogden Valley'    ,  57057],
    ['Ogden Valley'    ,  57058],
    ['Ogden Valley'    ,  57059],
    ['Ogden Valley'    ,  57060],
    ['Ogden Valley'    ,  57061],
    ['Ogden Valley'    ,  57062],
    ['Ogden Valley'    ,  57063],
    ['Ogden Valley'    ,  57064],
    ['Ogden Valley'    ,  57065],
    ['Ogden Valley'    ,  57066],
    ['Eden'            ,  57067],
    ['Eden'            ,  57068],
    ['Eden'            ,  57069],
    ['Eden'            ,  57070],
    ['Eden'            ,  57071],
    ['Ogden Valley'    ,  57072],
    ['Ogden Valley'    ,  57073],
    ['Ogden Valley'    ,  57074],
    ['Ogden Valley'    ,  57075],
    ['Ogden Valley'    ,  57076],
    ['Ogden Valley'    ,  57077],
    ['Ogden Valley'    ,  57078],
    ['Ogden Valley'    ,  57079],
    ['Ogden Valley'    ,  57080],
    ['Ogden Valley'    ,  57081],
    ['Ogden Valley'    ,  57082],
    ['Ogden Valley'    ,  57083],
    ['Ogden Valley'    ,  57084],
    ['Ogden Valley'    , 570141],
    ['Ogden Valley'    , 570142],
    ['Ogden Valley'    , 570153],
    ['Ogden Valley'    , 570155],
    ['Ogden Valley'    , 570156],
    ['Ogden Valley'    , 570157],
    ['Ogden Valley'    , 570166],
    ['Ogden Valley'    , 570167],
    ['Ogden Valley'    , 570168],
    ['Ogden Valley'    , 570169],
    ['Ogden Valley'    , 570232],
    ['Ogden Valley'    , 570233],
    ['Ogden Valley'    , 570234],
    ['Ogden Valley'    , 570235],
    ['Ogden Valley'    , 570236],
    ['Ogden Valley'    , 570237],
    ['Ogden Valley'    , 570239],
    ['Ogden Valley'    , 570240],
    ['Ogden Valley'    , 570241],
    ['Ogden Valley'    , 570243],
    ['Ogden Valley'    , 570250],
    ['Ogden Valley'    , 570251],
    ['Ogden Valley'    , 570252],
    ['Ogden Valley'    , 570253],
    ['Plain City'      , 570143],
    ['Plain City'      , 570147],
    ['Plain City'      , 570149],
    ['Farr West'       , 570194],
    ['North Ogden'     , 570192],
    ['Ogden'           , 570313],
    ['Ogden'           , 570261],
    ['South Ogden'     , 570380],
    ['South Ogden'     , 570391],
    ['Uintah Highlands', 570419],
    ['Uintah Highlands', 570420]
]
dfTAZCityEdits = pd.DataFrame(dTAZCityEdits, columns = ['CityOverride','CO_TAZID'])
display(dfTAZCityEdits)


,CityOverride,CO_TAZID
0,Ogden Valley,57001
1,Ogden Valley,57002
2,Ogden Valley,57003
3,Ogden Valley,57004
4,Ogden Valley,57005
...,...,...
114,Ogden,570261
115,South Ogden,570380
116,South Ogden,570391
117,Uintah Highlands,570419


In [6]:
# Make manual modifications to 

dfATOwithManual = pd.DataFrame.merge(dfATO,dfTAZCityEdits,on='CO_TAZID',how='left')
dfATOwithManual['CityOverride'] = dfATOwithManual['CityOverride'].fillna('')

dfATOwithManual['FinalCityName'] = dfATOwithManual['CITY_NAME'] 
dfATOwithManual.loc[dfATOwithManual['CityOverride']!='', 'FinalCityName'] = dfATOwithManual['CityOverride'] 

display(dfATOwithManual.groupby(['CityOverride' ]).agg(CNTREC=('CO_TAZID','count')))
display(dfATOwithManual.groupby(['FinalCityName']).agg(CNTREC=('CO_TAZID','count')))

,CNTREC
CityOverride,
,245
Eden,8
Farr West,1
Liberty,11
North Ogden,1
Ogden,2
Ogden Valley,86
Plain City,3
South Ogden,2


,CNTREC
FinalCityName,
Eden,8
Farr West,7
Harrisville,4
Hooper,9
Liberty,11
Marriott-Slaterville,10
North Ogden,16
Ogden,105
Ogden Valley,86


In [7]:
# Calcualte averages for each city
#display(dfATOwithManual.groupby(['FinalCityName']).agg(CNTREC=('CO_TAZID','count'),))
dfATOCalcs = dfATOwithManual[['FinalCityName','CO_TAZID','AUTO_JB','AUTO_JB_1','TRAN_JB']].copy()

# use zeros where no transit jobs
dfATOCalcs['TRAN_JB'  ].fillna(0                    , inplace=True)

# AUTO_JB_1 is WF TDM, so wherever there is NA us USTM
dfATOCalcs['AUTO_JB_1'].fillna(dfATOCalcs['AUTO_JB'], inplace=True)

display(dfATOCalcs)

dfATOCalcs = dfATOCalcs[['FinalCityName','CO_TAZID','AUTO_JB_1','TRAN_JB']]

dfATOCalcs.columns = ('CityName','CO_TAZID','AUTO_JB','TRAN_JB')

dfATOCalcs.to_csv('..\\intermediate\\atocalcs.csv')

dfATOCalcs

,FinalCityName,CO_TAZID,AUTO_JB,AUTO_JB_1,TRAN_JB
0,Plain City,570144,77215,74352.0,0.0
1,Plain City,570143,64942,62556.0,0.0
2,Plain City,570146,80559,78115.0,0.0
3,Plain City,570149,97162,94068.0,0.0
4,Farr West,570150,91894,88455.0,0.0
...,...,...,...,...,...
359,Ogden Valley,57079,2189,2189.0,0.0
360,Ogden Valley,57080,6160,6160.0,0.0
361,Ogden Valley,57082,3773,3773.0,0.0
362,Ogden Valley,57083,1996,1996.0,0.0


,CityName,CO_TAZID,AUTO_JB,TRAN_JB
0,Plain City,570144,74352.0,0.0
1,Plain City,570143,62556.0,0.0
2,Plain City,570146,78115.0,0.0
3,Plain City,570149,94068.0,0.0
4,Farr West,570150,88455.0,0.0
...,...,...,...,...
359,Ogden Valley,57079,2189.0,0.0
360,Ogden Valley,57080,6160.0,0.0
361,Ogden Valley,57082,3773.0,0.0
362,Ogden Valley,57083,1996.0,0.0


In [8]:
dfATOCalcs_CityMax = dfATOCalcs.groupby(['CityName'],as_index=False).agg(CNTREC=('CO_TAZID','count'),AUTO_JB_max=('AUTO_JB','max'),TRAN_JB_max=('TRAN_JB','max'))
dfATOCalcs_CityMax

,CityName,CNTREC,AUTO_JB_max,TRAN_JB_max
0,Eden,8,28750.0,0.0
1,Farr West,7,109537.0,6550.0
2,Harrisville,4,102566.0,7916.0
3,Hooper,9,90291.0,0.0
4,Liberty,11,25160.0,0.0
5,Marriott-Slaterville,10,124283.0,6093.0
6,North Ogden,16,92232.0,4627.0
7,Ogden,105,140566.0,27584.0
8,Ogden Valley,86,119186.0,0.0
9,Plain City,12,94068.0,0.0


In [9]:
dfATOCalcs_PercentMax = pd.DataFrame.merge(dfATOCalcs, dfATOCalcs_CityMax, on='CityName')
dfATOCalcs_PercentMax['AUTO_JB_percentCityMax'] = dfATOCalcs_PercentMax['AUTO_JB'] / dfATOCalcs_PercentMax['AUTO_JB_max']
dfATOCalcs_PercentMax['TRAN_JB_percentCityMax'] = dfATOCalcs_PercentMax['TRAN_JB'] / dfATOCalcs_PercentMax['TRAN_JB_max']
dfATOCalcs_PercentMax = dfATOCalcs_PercentMax.fillna(0)
dfATOCalcs_PercentMax

,CityName,CO_TAZID,AUTO_JB,TRAN_JB,CNTREC,AUTO_JB_max,TRAN_JB_max,AUTO_JB_percentCityMax,TRAN_JB_percentCityMax
0,Plain City,570144,74352.0,0.0,12,94068.0,0.0,0.790407,0.0
1,Plain City,570143,62556.0,0.0,12,94068.0,0.0,0.665008,0.0
2,Plain City,570146,78115.0,0.0,12,94068.0,0.0,0.830410,0.0
3,Plain City,570149,94068.0,0.0,12,94068.0,0.0,1.000000,0.0
4,Plain City,570145,78743.0,0.0,12,94068.0,0.0,0.837086,0.0
...,...,...,...,...,...,...,...,...,...
359,Eden,57035,28741.0,0.0,8,28750.0,0.0,0.999687,0.0
360,Eden,57067,25947.0,0.0,8,28750.0,0.0,0.902504,0.0
361,Eden,57068,24567.0,0.0,8,28750.0,0.0,0.854504,0.0
362,Eden,57071,23524.0,0.0,8,28750.0,0.0,0.818226,0.0


In [10]:
dfATOCalcs_PercentMax

,CityName,CO_TAZID,AUTO_JB,TRAN_JB,CNTREC,AUTO_JB_max,TRAN_JB_max,AUTO_JB_percentCityMax,TRAN_JB_percentCityMax
0,Plain City,570144,74352.0,0.0,12,94068.0,0.0,0.790407,0.0
1,Plain City,570143,62556.0,0.0,12,94068.0,0.0,0.665008,0.0
2,Plain City,570146,78115.0,0.0,12,94068.0,0.0,0.830410,0.0
3,Plain City,570149,94068.0,0.0,12,94068.0,0.0,1.000000,0.0
4,Plain City,570145,78743.0,0.0,12,94068.0,0.0,0.837086,0.0
...,...,...,...,...,...,...,...,...,...
359,Eden,57035,28741.0,0.0,8,28750.0,0.0,0.999687,0.0
360,Eden,57067,25947.0,0.0,8,28750.0,0.0,0.902504,0.0
361,Eden,57068,24567.0,0.0,8,28750.0,0.0,0.854504,0.0
362,Eden,57071,23524.0,0.0,8,28750.0,0.0,0.818226,0.0


In [11]:
dfCheck = dfATOCalcs_PercentMax.groupby(['CityName'],as_index=False).agg(NumTAZ=('CO_TAZID','count'),AutoMaxScore=('AUTO_JB_percentCityMax','max'),TranMaxScore=('TRAN_JB_percentCityMax','max'),AutoMinScore=('AUTO_JB_percentCityMax','min'),TranMinScore=('TRAN_JB_percentCityMax','min'))
dfCheck

,CityName,NumTAZ,AutoMaxScore,TranMaxScore,AutoMinScore,TranMinScore
0,Eden,8,1.0,0.0,0.818226,0.000000
1,Farr West,7,1.0,1.0,0.807535,0.000000
2,Harrisville,4,1.0,1.0,0.873808,0.353082
3,Hooper,9,1.0,0.0,0.630218,0.000000
4,Liberty,11,1.0,0.0,0.737043,0.000000
5,Marriott-Slaterville,10,1.0,1.0,0.758704,0.000000
6,North Ogden,16,1.0,1.0,0.612412,0.000000
7,Ogden,105,1.0,1.0,0.605552,0.000000
8,Ogden Valley,86,1.0,0.0,0.000000,0.000000
9,Plain City,12,1.0,0.0,0.665008,0.000000


In [12]:
dfATOCalcs_Scores = dfATOCalcs_PercentMax[['CO_TAZID','AUTO_JB_percentCityMax','TRAN_JB_percentCityMax']]
dfATOCalcs_Scores.columns = ('CO_TAZID','ATO_Auto_Score','ATO_Tran_Score')
dfATOCalcs_Scores

,CO_TAZID,ATO_Auto_Score,ATO_Tran_Score
0,570144,0.790407,0.0
1,570143,0.665008,0.0
2,570146,0.830410,0.0
3,570149,1.000000,0.0
4,570145,0.837086,0.0
...,...,...,...
359,57035,0.999687,0.0
360,57067,0.902504,0.0
361,57068,0.854504,0.0
362,57071,0.818226,0.0


In [13]:
dfATOCalcs_Scores.to_csv(os.path.join(dirIntermediate,'ATOScores_COTAZID.csv'),index=False)
print("export done")

export done


In [15]:
# join TAZ to ATO Scores
 
lyrTAZ = m.listLayers("TAZTemp")[0]

lyrTAZ.definitionQuery = ""

# DONE MANUALLY SINCE PROGRAMATIC DIDN'T WORK
#arcpy.management.Copy(os.path.join(dirIntermediate,'ATOScores_COTAZID.csv'), "ATOScores_COTAZID")

#arcpy.management.AddJoin("TAZTemp", "CO_TAZID", "ATOScores_COTAZID", "CO_TAZID")

#arcpy.management.CopyFeatures("TAZTemp", "TAZwithATOScores")

#arcpy.management.RemoveJoin("TAZTemp")

lyrTAZ.definitionQuery = ""

# delete unneeded fields
fldlist = arcpy.ListFields("TAZwithATOScores")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','CO_TAZID','ATO_Auto_Score','ATO_Tran_Score','OBJECTID_1','ATOScores_COTAZID_CO_TAZID','ATOScores_COTAZID_ATO_Auto_Score','ATOScores_COTAZID_ATO_Tran_Score','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("TAZwithATOScores",[fld.name])

print("DONE!")

deleting field: TAZ_TAZID
deleting field: TAZ_SA_TAZID
deleting field: TAZ_CO_TAZID
deleting field: TAZ_SUBAREAID
deleting field: TAZ_EXTERNAL
deleting field: TAZ_ACRES
deleting field: TAZ_DEVACRES
deleting field: TAZ_X
deleting field: TAZ_Y
deleting field: TAZ_ADJ_XY
deleting field: TAZ_CO_FIPS
deleting field: TAZ_CO_NAME
deleting field: TAZ_DISTSUPER
deleting field: TAZ_DSUP_NAME
deleting field: TAZ_DISTLRG
deleting field: TAZ_DLRG_NAME
deleting field: TAZ_DISTMED
deleting field: TAZ_DMED_NAME
deleting field: TAZ_DISTSML
deleting field: TAZ_DSML_NAME
deleting field: TAZ_CITY_NAME
deleting field: TAZ_IMX_GENRL
deleting field: TAZ_IMX_COALIX
deleting field: TAZ_IMX_OLGAIX
deleting field: TAZ_IMX_OLGAXI
deleting field: TAZ_IMX_PETRIX
deleting field: ATO19_USTM_csv_TAZID
deleting field: ATO19_USTM_csv_CO_TAZID
deleting field: ATO19_USTM_csv_DEVACRES
deleting field: ATO19_USTM_csv_HH
deleting field: ATO19_USTM_csv_JOB
deleting field: ATO19_USTM_csv_AUTO_JB
deleting field: ATO19_USTM_csv_A

In [49]:
# Geolocate Day Care Centers
#arcpy.agrcgeocoding.GeocodeTable("AGRC-2B10133F988346 ", "'report - 2022-02-18T121554#372$'", "ID_NUMBER", "LOCATION_ADDRESS_ONE", "LOCATION_ZIP_CODE", r"E:\GitHub\Housing-Affordability-Weber-County\intermediate\daycare_georeference", 'PROJCS["NAD_1983_UTM_Zone_12N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-111.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision', "Address points and road centerlines")

AttributeError: module 'arcpy' has no attribute 'agrcgeocoding'

# Schools

In [73]:
# SCHOOLS

lyrSchools = m.listLayers("Schools")[0]

lyrSchools.definitionQuery = "EDTYPE = 'Regular Education' And SCHOOLTYPE NOT IN ('Public', 'Online School', 'Regional Campus', 'Private School', 'Online Private School', 'Online Charter School', 'Charter School')"

arcpy.management.CopyFeatures("Schools", "SchoolsRegPublic")


<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\SchoolsRegPublic'>

In [74]:
# HIGHER EDUCATION

lyrSchools = m.listLayers("Schools")[0]

lyrSchools.definitionQuery = "EDTYPE IN ('Higher Education')"

arcpy.management.CopyFeatures("Schools", "SchoolsHigherEd")

lyrSchools.definitionQuery = ""

In [12]:
# HIGHER EDUCATION

strATLines = "AT_lines"

# Calculate ATO Scores by City 
lyrATLines = m.listLayers(strATLines)[0]

lyrATLines.definitionQuery = "ProjType IN ('Shared Use Path','Protected Bike Lane')"

arcpy.management.CopyFeatures(strATLines, "ActiveTransportationFacilities")

lyrATLines.definitionQuery = ""

<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\ActiveTransportationFacilities_Buffer_1_00'>

# Transit

In [90]:
# LCL STOPS - 1/4 Miles

strLocalBusStops = "UTA_Stops_and_Most_Recent_Ridership"

lyrLocalBusStops = m.listLayers(strLocalBusStops)[0]

lyrLocalBusStops.definitionQuery = "Mode='Bus'"

arcpy.management.CopyFeatures(strLocalBusStops, "LocalBusStops")

lyrLocalBusStops.definitionQuery = ""

In [48]:
# BRT STOPS - 1/3 Miles

srtBRTStops = "BRT_Stops"

lyrBRTStops = m.listLayers(srtBRTStops)[0]

lyrBRTStops.definitionQuery = "Existing IN ('Current')"

arcpy.management.CopyFeatures(srtBRTStops, "BRTStops")

lyrBRTStops.definitionQuery = "Existing IN ('Future','Planned')"

arcpy.management.CopyFeatures(srtBRTStops, "BRTStops_Future")

lyrBRTStops.definitionQuery = ""

In [49]:
# Rail Stops - 1/3 mile

strLRTStops = "LightRail_Stations"

lyrLRTStops = m.listLayers(strLRTStops)[0]

lyrLRTStops.definitionQuery = "Status IN ('Current')"

arcpy.management.CopyFeatures(strLRTStops, "LRTStops")

lyrLRTStops.definitionQuery = "Status IN ('Future','Planned')"

arcpy.management.CopyFeatures(strLRTStops, "LRTStops_Future")

lyrLRTStops.definitionQuery = ""

In [50]:
# Rail Stops - 1/3 mile

strCRTStops = "CommuterRail_Stations"

lyrCRTStops = m.listLayers(strCRTStops)[0]

lyrCRTStops.definitionQuery = "Status IN ('Current')"

arcpy.management.CopyFeatures(strCRTStops, "CRTStops")

lyrCRTStops.definitionQuery = "Status IN ('Future','Planned')"

arcpy.management.CopyFeatures(strCRTStops, "CRTStops_Future")

lyrCRTStops.definitionQuery = ""

# Interchanges

In [94]:
strInterchanges = "Roads_FreewayExits" # 2019 RTP

lyrInterchanges = m.listLayers(strInterchanges)[0]

lyrInterchanges.definitionQuery = ""

arcpy.management.CopyFeatures(strInterchanges, "Interchanges")

lyrInterchanges.definitionQuery = ""



In [6]:
strRTPRoadwayPoints = "Roadway_points" # 2019 RTP

lyrRTPRoadwayPoints = m.listLayers(strRTPRoadwayPoints)[0]

lyrRTPRoadwayPoints.definitionQuery = "ProjType IN ('New Construction') AND Project LIKE '%Interchange%'"

arcpy.management.CopyFeatures(strRTPRoadwayPoints, "Interchanges_Future")

lyrRTPRoadwayPoints.definitionQuery = ""

# Active Transportation

In [62]:
# info: https://gis.utah.gov/bike-related-gis-data-resources/

#Coded Values	
#Code	Name
#1A	1A Cycle track, at-grade, protected with parking
#1B	1B Cycle track, protected with barrier
#1C	1C Cycle track, raised and curb separated (may be multiuse with peds)
#2A	2A Buffered bike lane
#2B	2B Bike lane
#3A	3A Shoulder bikeway
#3B	3B Marked shared roadway
#3C	3C Signed shared roadway
#1	1 Cycle track, unspecified 
#2	2 Bike lane, unspecified 
#3	3 Other bike route, unspecified
#PP	Parallel Bike Path, Paved
#PU	Parallel Bike Path, Unpaved
#UN	Unknown Category


# for roadway bike facilities, only use cycle tracks

strRoads = "Roads" # 2019 RTP

lyrRoads = m.listLayers(strRoads)[0]

lyrRoads.definitionQuery = "BIKE_L     IN ('1A', '1B', '1C', '1') Or BIKE_R     IN ('1A', '1B', '1C', '1')"

arcpy.management.CopyFeatures(strRoads, "ATCycleTracks")

lyrRoads.definitionQuery = "BIKE_PLN_L IN ('1A', '1B', '1C', '1') Or BIKE_PLN_R IN ('1A', '1B', '1C', '1')"

arcpy.management.CopyFeatures(strRoads, "ATCycleTracks_Future")

lyrRoads.definitionQuery = ""


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset Roads does not exist or is not supported
WARNING 000725: Output Feature Class: Dataset E:\GitHub\Housing-Affordability-Weber-County\gis\HousingAffordability.gdb\ATCycleTracks already exists.
Failed to execute (CopyFeatures).


In [2]:
# NOTE: MAY HAVE TO MOVE FEATURE LAYER OUT OF GROUP TO ACCESS IN CODE

# for trails and paths use Class "Path"

strTrailsandPathways = "TrailsandPathways" # 2019 RTP

lyrTrailsandPathways = m.listLayers(strTrailsandPathways)[0]

lyrTrailsandPathways.definitionQuery = "Class='Path' AND Status='EXISTING'"

arcpy.management.CopyFeatures(strTrailsandPathways, "ATPaths")

lyrTrailsandPathways.definitionQuery = "Class='Path' AND Status='PROPOSED'"

arcpy.management.CopyFeatures(strTrailsandPathways, "ATPaths_Future")

lyrTrailsandPathways.definitionQuery = ""

# Amenitities


In [17]:
strGrocery = "Utah_Grocery_And_Food_Stores__UDAF_.shp"

lyrGrocery = m.listLayers(strGrocery)[0]

lyrGrocery.definitionQuery = "TYPE NOT IN ('Convenience Grocery','Salvage Grocery')"

arcpy.management.CopyFeatures(strGrocery, "GroceryStores")

lyrGrocery.definitionQuery = ""

In [3]:
strHealthCare = "HealthCareFacilities"

lyrHealthCare = m.listLayers(strHealthCare)[0]

lyrHealthCare.definitionQuery = "TYPE IN ('CLINIC', 'HOSPITAL', 'URGENT CARE CLINIC')"

arcpy.management.CopyFeatures(strHealthCare, "HealthCare")

lyrHealthCare.definitionQuery = ""


In [17]:
strWalkAccessToParks = "WalkAccesstoParks_10Min_Polygons.shp"

lyrWalkAccessToParks = m.listLayers(strGrocery)[0]

lyrWalkAccessToParks.definitionQuery = ""

arcpy.management.CopyFeatures(strWalkAccessToParks, "ParksAndOpenSpace")

lyrWalkAccessToParks.definitionQuery = ""

# delete unneeded fields
fldlist = arcpy.ListFields("ParksAndOpenSpace")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','FacilityID','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("ParksAndOpenSpace",[fld.name])

print("DONE!")

deleting field: FID_1
deleting field: Name
deleting field: FromBreak
deleting field: ToBreak
deleting field: SHAPE_Leng
DONE!


# Ring Buffers

In [95]:
arcpy.analysis.MultipleRingBuffer("Interchanges"                  , "Interchanges_Buffers"                  , [1/2, 1, 2], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("Interchanges_Future"           , "Interchanges_Future_Buffers"           , [1/2, 1, 2], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("LocalBusStops"                 , "LocalBusStops_Buffers"                 , [1/4      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("BRTStops"                      , "BRTStops_Buffers"                      , [1/3      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("BRTStops_Future"               , "BRTStops_Future_Buffers"               , [1/3      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("LRTStops"                      , "LRTStops_Buffers"                      , [1/3      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("LRTStops_Future"               , "LRTStops_Future_Buffers"               , [1/3      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("CRTStops"                      , "CRTStops_Buffers"                      , [1/3      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("CRTStops_Future"               , "CRTStops_Future_Buffers"               , [1/3      ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("ChildCare"                     , "ChildCare_Buffers"                     , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("HealthCare"                    , "HealthCare_Buffers"                    , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("SchoolsRegPublic"              , "SchoolsRegPublic_Buffers"              , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("SchoolsHigherEd"               , "SchoolsHigherEd_Buffers"               , [     1, 2], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("GroceryStores"                 , "GroceryStores_Buffers"                 , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("CommunityCenters"              , "CommunityCenters_Buffers"              , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("ATPaths"                       , "ATPaths_Buffers"                       , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("ATPaths_Future"                , "ATPaths_Future_Buffers"                , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("ATCycleTracks"                 , "ATCycleTracks_Buffers"                 , [1/2, 1   ], "miles", "", "ALL")
arcpy.analysis.MultipleRingBuffer("ATCycleTracks_Future"          , "ATCycleTracks_Future_Buffers"          , [1/2, 1   ], "miles", "", "ALL")

<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\DayCareCenters_Buffers'>

# Centers Splitting

In [9]:
#

strCenters = "Centers"

lyrCenters = m.listLayers(strCenters)[0]

lyrCenters.definitionQuery = "AreaType = 'Metropolitan Center'"

arcpy.management.CopyFeatures(strCenters, "MetropolitanCenters")

lyrCenters.definitionQuery = "AreaType = 'Urban Center'"

rcpy.management.CopyFeatures(strCenters, "UrbanCenters")

lyrCenters.definitionQuery = "AreaType = 'City Center'"

arcpy.management.CopyFeatures(strCenters, "CityCenters")

lyrCenters.definitionQuery = "AreaType = 'Neighborhood Center'"

arcpy.management.CopyFeatures(strCenters, "NeighborhoodCenters")

lyrCenters.definitionQuery = ""

<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\MetropolitanCenters'>

# Qualified Opportunity Zones

In [45]:
strUQOZ = "UtahQualifiedOpportunityZones.shp"

lyrUQOZ = m.listLayers(strUQOZ)[0]

lyrUQOZ.definitionQuery = "COUNTYNAME='Weber'"

arcpy.management.CopyFeatures(strUQOZ, "UtahQualifiedOpportunityZones")

lyrUQOZ.definitionQuery = ""

# delete unneeded fields
fldlist = arcpy.ListFields("UtahQualifiedOpportunityZones")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','OBJECTID_1','QOZ','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("UtahQualifiedOpportunityZones",[fld.name])

print("DONE!")

deleting field: GEOID10
deleting field: STATENAME
deleting field: COUNTYNAME
deleting field: Population
deleting field: UnempRate
deleting field: PerCapInc
deleting field: EDD
deleting field: SHAPE_Leng
DONE!


# Parcels - West Weber

In [105]:
strParcels = "parcels_2019"

lyrParcels = m.listLayers(strParcels)[0]

lyrParcels.definitionQuery = "COUNTY_ID IN ('57')"

arcpy.management.CopyFeatures(strParcels, "parcels_2019_west_weber")

lyrParcels.definitionQuery = ""

# delete unneeded fields
fldlist = arcpy.ListFields("parcels_2019_west_weber")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','PARCEL_ID','building_type','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("parcels_2019_west_weber",[fld.name])

print("DONE!")

deleting field: parcel_id_REMM
deleting field: CO_NAME
deleting field: TAZID
deleting field: COUNTY_ID
deleting field: TOTAL_MKT_VALUE
deleting field: land_value
deleting field: residential_units
deleting field: building_sqft
deleting field: FLOORS_CNT
deleting field: year_built
deleting field: EFFBUILT_YR
deleting field: IS_OUG
deleting field: max_dua
deleting field: max_far
deleting field: max_height
deleting field: type1
deleting field: type2
deleting field: type3
deleting field: type4
deleting field: type5
deleting field: type6
deleting field: type7
deleting field: type8
deleting field: agriculture
deleting field: basebldg
deleting field: NoBuild
deleting field: redev_friction
deleting field: building_type_id
deleting field: x
deleting field: y
deleting field: parcel_acres
deleting field: Split
deleting field: Split_Factor
deleting field: parcel_id_REMM_old
deleting field: residential_price
deleting field: non_residential_price
deleting field: non_residential_sqft
deleting field: T

# Parcels - East Weber

In [102]:
arcpy.analysis.PairwiseDissolve("TAZ", "TAZ_Weber_Subareas", ["SUBAREAID", "CO_FIPS"])

lyrTAZWeberSubareas = m.listLayers("TAZ_Weber_Subareas")[0]

lyrTAZWeberSubareas.definitionQuery = "TAZ_SUBAREAID IN (0)"

arcpy.management.CopyFeatures("TAZ_Weber_Subareas", "TAZ_Weber_Subareas_NonWFTDM")

lyrTAZWeberSubareas.definitionQuery = ""

In [104]:
arcpy.analysis.Intersect("TAZ_Weber_Subareas_NonWFTDM #;Parcels_Weber #","parcels_east_weber", "ALL", None, "INPUT")

<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\parcels_east_weber'>

In [106]:

# delete unneeded fields
fldlist = arcpy.ListFields("parcels_east_weber")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','PARCEL_ID','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("parcels_east_weber",[fld.name])

print("DONE!")

deleting field: FID_TAZ_Weber_Subareas_NonWFTDM
deleting field: TAZ_SUBAREAID
deleting field: TAZ_CO_FIPS
deleting field: FID_Parcels_Weber
deleting field: FIPS
deleting field: PARCEL_ADD
deleting field: PARCEL_CITY
deleting field: PARCEL_ZIP
deleting field: OWN_TYPE
deleting field: RECORDER
deleting field: ParcelsCur
deleting field: ParcelsRec
deleting field: ParcelsPub
deleting field: ParcelYear
deleting field: ParcelNotes
deleting field: CoParcel_URL
DONE!


# Combine Parcels

In [107]:
arcpy.management.Merge(['parcels_2019_west_weber','parcels_east_weber'], 'parcels_weber')

<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\parcels_weber'>

# Communities

In [2]:
#arcpy.analysis.Union(["cb_2021_us_place_500k",
#                      "WeberCounty"],
#                     "WeberCommunities")

<Result 'E:\\GitHub\\Housing-Affordability-Weber-County\\gis\\HousingAffordability.gdb\\WeberCommunities'>

In [3]:
#
## delete unneeded fields
#fldlist = arcpy.ListFields("WeberCommunities")
#
#for fld in fldlist: 
#    if (fld.name not in ['OBJECTID','NAME','Shape','Shape_Length','Shape_Area']):
#        print("deleting field: " + fld.name)
#        arcpy.DeleteField_management("WeberCommunities",[fld.name])
#
#print("DONE!")

deleting field: FID_cb_2021_us_place_500k
deleting field: STATEFP
deleting field: PLACEFP
deleting field: PLACENS
deleting field: AFFGEOID
deleting field: GEOID
deleting field: NAMELSAD
deleting field: STUSPS
deleting field: STATE_NAME
deleting field: LSAD
deleting field: ALAND
deleting field: AWATER
deleting field: FID_WeberCounty
deleting field: COUNTYNBR
DONE!


In [13]:
#manually create communities

# Union with DataSets

In [16]:
arcpy.analysis.Union(["parcels_weber",
                      "WeberCommunities",
                      "Centers",
                      "TAZwithATOScores",
                      "CRTStops_Buffers",
                      "CRTStops_Future_Buffers",
                      "LRTStops_Buffers",
                      "LRTStops_Future_Buffers",
                      "BRTStops_Buffers",
                      "BRTStops_Future_Buffers",
                      "LocalBusStops_Buffers",
                      "Interchanges_Buffers",
                      "Interchanges_Future_Buffers",
                      "ATPaths_Buffers",
                      "ATPaths_Future_Buffers",
                      "ATCycleTracks_Buffers",
                      "ATCycleTracks_Future_Buffers",
                      "ChildCare_Buffers",
                      "HealthCare_Buffers",
                      "SchoolsRegPublic_Buffers",
                      "SchoolsHigherEd_Buffers",
                      "GroceryStores_Buffers",
                      "ParksAndOpenSpace",
                      "CommunityCenters_Buffers",
                      "UtahQualifiedOpportunityZones"],
                     "UNIONED")


<Result 'E:\\GitHub\\Housing-Suitability-Weber-County\\gis\\HousingAffordability.gdb\\UNIONED'>

# Clip to County

In [4]:
strCounty = "Utah_County_Boundaries"

lyrCounty = m.listLayers(strCounty)[0]

lyrCounty.definitionQuery = "NAME='WEBER'"

arcpy.management.CopyFeatures(strCounty, "WeberCounty")

lyrCounty.definitionQuery = ""

# delete unneeded fields
fldlist = arcpy.ListFields("WeberCounty")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID_1','COUNTYNBR','Shape','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("WeberCounty",[fld.name])

print("DONE!")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset Utah_County_Boundaries does not exist or is not supported
WARNING 000725: Output Feature Class: Dataset E:\GitHub\Housing-Suitability-Weber-County\gis\HousingAffordability.gdb\WeberCounty already exists.
Failed to execute (CopyFeatures).


In [7]:
arcpy.analysis.Intersect(["WeberCounty","UNIONED"], "UNIONED_WEBER")

lyrUnionedWeber = m.listLayers("UNIONED_WEBER")[0]

# filter out any shapes that didn't have an underlying parcel or communitity
lyrUnionedWeber.definitionQuery = "FID_parcels_weber<>-1 AND FID_WeberCommunities<>-1"

arcpy.management.CopyFeatures("UNIONED_WEBER", "UNIONED_WEBER_FILTERED")

lyrUnionedWeber.definitionQuery = ""

In [8]:
myFCls = "UNIONED_WEBER_FILTERED"
arcpy.AddField_management(myFCls, "UNIQ_ID", "LONG")
dataDict = {}
i = 1
updateRows = arcpy.da.UpdateCursor(myFCls, ["OBJECTID", "UNIQ_ID"])
for updateRow in updateRows:
    if updateRow[0] not in dataDict:
        dataDict[updateRow[0]] = i
        i+=1
    updateRow[1] = dataDict[updateRow[0]]
    updateRows.updateRow(updateRow)
del updateRow, updateRows

In [9]:
# filter out any shapes that didn't have an underlying parcel
lyrUnionedWeber.definitionQuery = "FID_parcels_weber<>-1"

arcpy.management.CopyFeatures("UNIONED_WEBER_FILTERED", "ParcelPartsIDsOnly")

lyrUnionedWeber.definitionQuery = ""

# delete unneeded fields
fldlist = arcpy.ListFields("ParcelPartsIDsOnly")

for fld in fldlist: 
    if (fld.name not in ['OBJECTID','UNIQ_ID','Shape','CommCode','Shape_Length','Shape_Area']):
        print("deleting field: " + fld.name)
        arcpy.DeleteField_management("ParcelPartsIDsOnly",[fld.name])

        

deleting field: FID_WeberCounty
deleting field: COUNTYNBR
deleting field: FID_UNIONED
deleting field: FID_parcels_weber
deleting field: PARCEL_ID
deleting field: building_type
deleting field: FID_Centers
deleting field: AreaType
deleting field: FID_TAZwithATOScores
deleting field: OBJECTID_12
deleting field: CO_TAZID
deleting field: ATO_Auto_Score
deleting field: ATO_Tran_Score
deleting field: FID_CRTStops_Buffers
deleting field: distance
deleting field: FID_CRTStops_Future_Buffers
deleting field: distance_1
deleting field: FID_LRTStops_Buffers
deleting field: distance_12
deleting field: FID_LRTStops_Future_Buffers
deleting field: distance_12_13
deleting field: FID_BRTStops_Buffers
deleting field: distance_12_13_14
deleting field: FID_BRTStops_Future_Buffers
deleting field: distance_12_13_14_15
deleting field: FID_LocalBusStops_Buffers
deleting field: distance_12_13_14_15_16
deleting field: FID_Interchanges_Buffers
deleting field: distance_12_13_14_15_16_17
deleting field: FID_Intercha

In [11]:
# export to csv
arcpy.conversion.TableToTable("UNIONED_WEBER_FILTERED", "E:\\GitHub\\Housing-Suitability-Weber-County\\intermediate", "unioned_weber_filtered.csv")

<Result 'E:\\GitHub\\Housing-Suitability-Weber-County\\intermediate\\unioned_weber_filtered2.csv'>

# Read in Unioned Table for DataFrame processing

In [12]:
dfUnionedWeber = pd.read_csv("E:\\GitHub\\Housing-Suitability-Weber-County\\intermediate\\unioned_weber_filtered.csv",low_memory=False)
dfUnionedWeber

,OID_,FID_WeberCounty,COUNTYNBR,FID_UNIONED,FID_parcels_weber,PARCEL_ID,building_type,FID_Centers,AreaType,FID_TAZwithATOScores,OBJECTID_12,CO_TAZID,ATO_Auto_Score,ATO_Tran_Score,FID_CRTStops_Buffers,distance,FID_CRTStops_Future_Buffers,distance_1,FID_LRTStops_Buffers,distance_12,FID_LRTStops_Future_Buffers,distance_12_13,FID_BRTStops_Buffers,distance_12_13_14,FID_BRTStops_Future_Buffers,distance_12_13_14_15,FID_LocalBusStops_Buffers,distance_12_13_14_15_16,FID_Interchanges_Buffers,distance_12_13_14_15_16_17,FID_Interchanges_Future_Buffers,distance_12_13_14_15_16_17_18,FID_ATPaths_Buffers,distance_12_13_14_15_16_17_18_19,FID_ATPaths_Future_Buffers,distance_12_13_14_15_16_17_18_19_20,FID_ATCycleTracks_Buffers,distance_12_13_14_15_16_17_18_19_20_21,FID_ATCycleTracks_Future_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22,FID_ChildCare_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23,FID_HealthCare_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24,FID_SchoolsRegPublic_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25,FID_SchoolsHigherEd_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26,FID_GroceryStores_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27,FID_ParksAndOpenSpace,FacilityID,FID_CommunityCenters_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28,FID_UtahQualifiedOpportunityZones,OBJECTID_12_13,QOZ,FID_WeberCommunities,FID_WeberCommunities_1,NAME,CommCode,FID_UintahHighlands,Shape_Length,Shape_Area,UNIQ_ID
0,1,1,29,63,2870,160040011,Governmental,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.002262,8.412697e-08,1
1,2,1,29,72,11906,160120011,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003783,4.325147e-07,2
2,3,1,29,73,11907,160120012,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.007786,1.099263e-06,3
3,4,1,29,74,11912,160120029,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003640,3.852401e-07,4
4,5,1,29,75,11913,160120030,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003219,3.821471e-07,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147064,147065,1,29,183231,27479,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,-1,0.0,1,0.5,2,1.0,-1,0.0,-1,0.0,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147065
147065,147066,1,29,183232,27480,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,-1,0.0,1,0.5,2,1.0,-1,0.0,-1,0.0,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147066
147066,147067,1,29,192694,1246,020300004,Governmental,-1,NaN,206,204,570343,0.778922,0.400341,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,1,0.333333,1,0.25,-1,0.0,-1,0.0,1,0.5,2,1.0,2,1.0,1,0.5,1,0.5,1,0.5,2,1.0,2,2.0,1,0.5,1,0,2,1.0,4,31,DESIGNATEDQOZ,15,32196,Ogden,OG,-1,0.000011,6.620000e-13,147

In [13]:
# create base scoring table

#dfScores = dfUnionedWeber

In [14]:
# Centers Scores

strCenterColName = 'AreaType'

dfCenters = pd.DataFrame({'center_code':['CM'                 ,'CU'          ,'CC'         ,'CN'                 ],
                          'center_name':['Metropolitan Center','Urban Center','City Center','Neighborhood Center']})
display(dfCenters)

for index, row in dfCenters.iterrows():
    _cname = row['center_name']
    _ccode = row['center_code']
    print(_cname)
    
    dfUnionedWeber[_ccode] = 0
    
    dfUnionedWeber.loc[(dfUnionedWeber[strCenterColName]==_cname),_ccode] = 1
    
# sum for check
display(dfUnionedWeber[['CM','CU','CC','CN']].sum())
display(dfUnionedWeber[['CM','CU','CC','CN']].max())

,center_code,center_name
0,CM,Metropolitan Center
1,CU,Urban Center
2,CC,City Center
3,CN,Neighborhood Center


Metropolitan Center
Urban Center
City Center
Neighborhood Center


CM    2589
CU       0
CC    9263
CN    2353
dtype: int64

CM    1
CU    0
CC    1
CN    1
dtype: int64

In [15]:
# ATO Scores
dfUnionedWeber['AA'] = dfUnionedWeber['ATO_Auto_Score']
dfUnionedWeber['AT'] = dfUnionedWeber['ATO_Tran_Score']

# sum for check
display(dfUnionedWeber[['AA','AT']].sum())
display(dfUnionedWeber[['AA','AT']].max())

AA    114564.611943
AT     50067.741581
dtype: float64

AA    1.0
AT    1.0
dtype: float64

In [16]:
# Transportation Scores

dfUnionedWeber['TT'] = 0
# Transit only has single buffers
# local bus get 1/2 score, all other gets full score
dfUnionedWeber.loc[ (dfUnionedWeber['FID_LocalBusStops_Buffers'  ]!=-1) ,'TT'] = 0.5
# local bus values are overwritten by other modes
dfUnionedWeber.loc[((dfUnionedWeber['FID_BRTStops_Buffers'       ]!=-1) |
                    (dfUnionedWeber['FID_BRTStops_Future_Buffers']!=-1) |
                    (dfUnionedWeber['FID_LRTStops_Buffers'       ]!=-1) |
                    (dfUnionedWeber['FID_LRTStops_Future_Buffers']!=-1) |
                    (dfUnionedWeber['FID_CRTStops_Buffers'       ]!=-1) |
                    (dfUnionedWeber['FID_CRTStops_Future_Buffers']!=-1)),'TT'] = 1


# Freeway Access
dfFreewayAccess = pd.DataFrame({'tempscore' :['TF1'                       ,'TF2'                          ],
                                'moe'       :['Interchanges_Buffers'      ,'Interchanges_Future_Buffers'  ],
                                'fieldname' :['distance_12_13_14_15_16_17','distance_12_13_14_15_16_17_18'],
                                'smallest'  :[0.5                         ,0.5                            ]})
                                # smallest buffer
display(dfFreewayAccess)

# future

for index, row in dfFreewayAccess.iterrows():
    dfUnionedWeber[row['tempscore']] = 1 / (dfUnionedWeber[row['fieldname']] * (1 / row['smallest'])) # equation insures smallest buffer score is always 1
    dfUnionedWeber[row['tempscore']].replace([np.inf, -np.inf], 0, inplace=True)
    
dfUnionedWeber['TF'] = dfUnionedWeber[dfFreewayAccess['tempscore'].tolist()].values.max(1)

# Active Transportation Access 
dfATAccess = pd.DataFrame({'tempscore' :['TA1'                             ,'TA2'                                ,'TA3'                                   ,'TA4'                                      ],
                           'moe'       :['ATPaths_Buffers'                 ,'ATPaths_Future_Buffers'             ,'ATCycleTracks_Buffers'                 ,'ATCycleTracks_Future_Buffers'             ],
                           'fieldname' :['distance_12_13_14_15_16_17_18_19','distance_12_13_14_15_16_17_18_19_20','distance_12_13_14_15_16_17_18_19_20_21','distance_12_13_14_15_16_17_18_19_20_21_22'],
                           'smallest'  :[0.5                               ,0.5                                  ,0.5                                     ,0.5                                        ]})
                            #multiplier is equal to 1 / the largest buffer distance
display(dfATAccess)

for index, row in dfATAccess.iterrows():
    dfUnionedWeber[row['tempscore']] = 1 / (dfUnionedWeber[row['fieldname']] * (1 / row['smallest'])) # equation insures smallest buffer score is always 1
    dfUnionedWeber[row['tempscore']].replace([np.inf, -np.inf], 0, inplace=True)

dfUnionedWeber['TA'] = dfUnionedWeber[dfATAccess['tempscore'].tolist()].values.max(1)

# sum for check
display(dfUnionedWeber[['TT','TF1','TF2','TF','TA']].sum())
display(dfUnionedWeber[['TT','TF1','TF2','TF','TA']].max())


,tempscore,moe,fieldname,smallest
0,TF1,Interchanges_Buffers,distance_12_13_14_15_16_17,0.5
1,TF2,Interchanges_Future_Buffers,distance_12_13_14_15_16_17_18,0.5


,tempscore,moe,fieldname,smallest
0,TA1,ATPaths_Buffers,distance_12_13_14_15_16_17_18_19,0.5
1,TA2,ATPaths_Future_Buffers,distance_12_13_14_15_16_17_18_19_20,0.5
2,TA3,ATCycleTracks_Buffers,distance_12_13_14_15_16_17_18_19_20_21,0.5
3,TA4,ATCycleTracks_Future_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22,0.5


TT      32556.5
TF1     14801.5
TF2       368.5
TF      14801.5
TA     104578.5
dtype: float64

TT     1.00
TF1    1.00
TF2    0.25
TF     1.00
TA     1.00
dtype: float64

In [17]:
dfUnionedWeber

,OID_,FID_WeberCounty,COUNTYNBR,FID_UNIONED,FID_parcels_weber,PARCEL_ID,building_type,FID_Centers,AreaType,FID_TAZwithATOScores,OBJECTID_12,CO_TAZID,ATO_Auto_Score,ATO_Tran_Score,FID_CRTStops_Buffers,distance,FID_CRTStops_Future_Buffers,distance_1,FID_LRTStops_Buffers,distance_12,FID_LRTStops_Future_Buffers,distance_12_13,FID_BRTStops_Buffers,distance_12_13_14,FID_BRTStops_Future_Buffers,distance_12_13_14_15,FID_LocalBusStops_Buffers,distance_12_13_14_15_16,FID_Interchanges_Buffers,distance_12_13_14_15_16_17,FID_Interchanges_Future_Buffers,distance_12_13_14_15_16_17_18,FID_ATPaths_Buffers,distance_12_13_14_15_16_17_18_19,FID_ATPaths_Future_Buffers,distance_12_13_14_15_16_17_18_19_20,FID_ATCycleTracks_Buffers,distance_12_13_14_15_16_17_18_19_20_21,FID_ATCycleTracks_Future_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22,FID_ChildCare_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23,FID_HealthCare_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24,FID_SchoolsRegPublic_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25,FID_SchoolsHigherEd_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26,FID_GroceryStores_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27,FID_ParksAndOpenSpace,FacilityID,FID_CommunityCenters_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28,FID_UtahQualifiedOpportunityZones,OBJECTID_12_13,QOZ,FID_WeberCommunities,FID_WeberCommunities_1,NAME,CommCode,FID_UintahHighlands,Shape_Length,Shape_Area,UNIQ_ID,CM,CU,CC,CN,AA,AT,TT,TF1,TF2,TF,TA1,TA2,TA3,TA4,TA
0,1,1,29,63,2870,160040011,Governmental,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.002262,8.412697e-08,1,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,2,1,29,72,11906,160120011,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003783,4.325147e-07,2,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,3,1,29,73,11907,160120012,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.007786,1.099263e-06,3,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,4,1,29,74,11912,160120029,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003640,3.852401e-07,4,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,5,1,29,75,11913,160120030,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003219,3.821471e-07,5,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147064,147065,1,29,183231,27479,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,-1,0.0,1,0.5,2,1.0,-1,0.0,-1,0.0,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147065,0,0,0,0,0.857134,0.391064,0.5,0.25,0.0,0.25,0.5,0.5,0.0,0.5,0.5
147065,147066,1,29,183232,27480,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.39

# Amenities Scores

In [18]:
#score multipliers - match to buffers above - so that max is always 1.0
dfAmenities = pd.DataFrame({'scorename' :['AC'                                          ,'AH'                                             ,'AG'                                                      ,'AM'                                                         ],
                            'distfield' :['distance_12_13_14_15_16_17_18_19_20_21_22_23','distance_12_13_14_15_16_17_18_19_20_21_22_23_24','distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27','distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28'],
                            'smallest'  :[0.5                                           ,0.5                                              ,0.5                                                       ,0.5                                                          ]})

display(dfAmenities)

for index, row in dfAmenities.iterrows():
    dfUnionedWeber[row['scorename']] = 1 / (dfUnionedWeber[row['distfield']] * (1 / row['smallest']))
    dfUnionedWeber[row['scorename']].replace([np.inf, -np.inf], 0, inplace=True)

# Education Access
dfEducation = pd.DataFrame({'tempscore' :['AE1'                                               ,'AE2'                                                  ],
                            'moe'       :['SchoolsRegPublic_Buffers'                          ,'SchoolsHigherEd_Buffers '                             ],
                            'fieldname' :['distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25','distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26'],
                            'smallest'  :[0.5                                                 ,1                                                      ]})
                            # smallest buffer size

display(dfEducation)

for index, row in dfEducation.iterrows():
    dfUnionedWeber[row['tempscore']] = 1 / (dfUnionedWeber[row['fieldname']] * (1 / row['smallest'])) # equation insures smallest buffer score is always 1
    dfUnionedWeber[row['tempscore']].replace([np.inf, -np.inf], 0, inplace=True)

dfUnionedWeber['AE'] = dfUnionedWeber[dfEducation['tempscore'].tolist()].values.max(1)


# Parks and Open Space

dfUnionedWeber['AP'] = 0
    
dfUnionedWeber.loc[(dfUnionedWeber['FID_ParksAndOpenSpace']==1),'AP'] = 1


# sum for check
display(dfUnionedWeber[['AC','AH','AE1','AE2','AE','AG','AP','AM']].sum())
display(dfUnionedWeber[['AC','AH','AE1','AE2','AE','AG','AP','AM']].max())

,scorename,distfield,smallest
0,AC,distance_12_13_14_15_16_17_18_19_20_21_22_23,0.5
1,AH,distance_12_13_14_15_16_17_18_19_20_21_22_23_24,0.5
2,AG,distance_12_13_14_15_16_17_18_19_20_21_22_23_2...,0.5
3,AM,distance_12_13_14_15_16_17_18_19_20_21_22_23_2...,0.5


,tempscore,moe,fieldname,smallest
0,AE1,SchoolsRegPublic_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_2...,0.5
1,AE2,SchoolsHigherEd_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_2...,1.0


AC     64268.0
AH     33855.5
AE1    67291.5
AE2    36124.0
AE     76422.5
AG     56289.5
AP     52564.0
AM     22577.5
dtype: float64

AC     1.0
AH     1.0
AE1    1.0
AE2    1.0
AE     1.0
AG     1.0
AP     1.0
AM     1.0
dtype: float64

In [19]:
dfUnionedWeber[dfEducation['tempscore'].tolist()].values.max(1)

array([0. , 0. , 0. , ..., 0.5, 0.5, 0.5])

In [20]:
dfUnionedWeber

,OID_,FID_WeberCounty,COUNTYNBR,FID_UNIONED,FID_parcels_weber,PARCEL_ID,building_type,FID_Centers,AreaType,FID_TAZwithATOScores,OBJECTID_12,CO_TAZID,ATO_Auto_Score,ATO_Tran_Score,FID_CRTStops_Buffers,distance,FID_CRTStops_Future_Buffers,distance_1,FID_LRTStops_Buffers,distance_12,FID_LRTStops_Future_Buffers,distance_12_13,FID_BRTStops_Buffers,distance_12_13_14,FID_BRTStops_Future_Buffers,distance_12_13_14_15,FID_LocalBusStops_Buffers,distance_12_13_14_15_16,FID_Interchanges_Buffers,distance_12_13_14_15_16_17,FID_Interchanges_Future_Buffers,distance_12_13_14_15_16_17_18,FID_ATPaths_Buffers,distance_12_13_14_15_16_17_18_19,FID_ATPaths_Future_Buffers,distance_12_13_14_15_16_17_18_19_20,FID_ATCycleTracks_Buffers,distance_12_13_14_15_16_17_18_19_20_21,FID_ATCycleTracks_Future_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22,...,FID_GroceryStores_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27,FID_ParksAndOpenSpace,FacilityID,FID_CommunityCenters_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28,FID_UtahQualifiedOpportunityZones,OBJECTID_12_13,QOZ,FID_WeberCommunities,FID_WeberCommunities_1,NAME,CommCode,FID_UintahHighlands,Shape_Length,Shape_Area,UNIQ_ID,CM,CU,CC,CN,AA,AT,TT,TF1,TF2,TF,TA1,TA2,TA3,TA4,TA,AC,AH,AG,AM,AE1,AE2,AE,AP
0,1,1,29,63,2870,160040011,Governmental,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.002262,8.412697e-08,1,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,1,29,72,11906,160120011,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003783,4.325147e-07,2,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,1,29,73,11907,160120012,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.007786,1.099263e-06,3,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,1,29,74,11912,160120029,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003640,3.852401e-07,4,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,1,29,75,11913,160120030,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0.0,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003219,3.821471e-07,5,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147064,147065,1,29,183231,27479,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,...,-1,0.0,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147065,0,0,0,0,0.857134,0.391064,0.5,0.25,0.0,0.25,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,0.5,0.5,0.0,0.5,1
147065,147066,1,29,183232,27480,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,...,-1,0.0,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147066,0,0,0,0,0.857134,0.391064,0.5,0.25,0.0,0.25,0

# Opportunity Zone

In [21]:
# Parks and Open Space

dfUnionedWeber['OZ'] = 0
    
dfUnionedWeber.loc[(dfUnionedWeber['FID_UtahQualifiedOpportunityZones']!=-1),'OZ'] = 1

# Building Code

In [22]:
dBuildingCodes= [
    ["AG","Agriculture"             ], 
    ["EM","Empty Buildable"         ],
    ["OS","Open Space Non-Buildable"],
    ["CH","Church"                  ],
    ["SF","Single Family Res"       ],
    ["MF","Multi Family Res"        ],
    ["GQ","Group Quarters"          ],
    ["GO","Governmental"            ],
    ["ED","Educational"             ],
    ["HE","Healthcare"              ],
    ["RE","Retail"                  ],
    ["OF","Office"                  ],
    ["IN","Industrial"              ],
    ["OT","Other"                   ],
    ["UT","Utilities"               ],
    ["NB","No Build"                ]
]

dfBuildingCodes = pd.DataFrame(dBuildingCodes, columns = ['BC','building_type'])
display(dfBuildingCodes)


dfUnionedWeberWithBC = pd.DataFrame.merge(dfUnionedWeber,dfBuildingCodes,on='building_type',how='left')
dfUnionedWeberWithBC['BC'] = dfUnionedWeberWithBC['BC'].fillna('NO')
dfUnionedWeberWithBC

,BC,building_type
0,AG,Agriculture
1,EM,Empty Buildable
2,OS,Open Space Non-Buildable
3,CH,Church
4,SF,Single Family Res
5,MF,Multi Family Res
6,GQ,Group Quarters
7,GO,Governmental
8,ED,Educational
9,HE,Healthcare


,OID_,FID_WeberCounty,COUNTYNBR,FID_UNIONED,FID_parcels_weber,PARCEL_ID,building_type,FID_Centers,AreaType,FID_TAZwithATOScores,OBJECTID_12,CO_TAZID,ATO_Auto_Score,ATO_Tran_Score,FID_CRTStops_Buffers,distance,FID_CRTStops_Future_Buffers,distance_1,FID_LRTStops_Buffers,distance_12,FID_LRTStops_Future_Buffers,distance_12_13,FID_BRTStops_Buffers,distance_12_13_14,FID_BRTStops_Future_Buffers,distance_12_13_14_15,FID_LocalBusStops_Buffers,distance_12_13_14_15_16,FID_Interchanges_Buffers,distance_12_13_14_15_16_17,FID_Interchanges_Future_Buffers,distance_12_13_14_15_16_17_18,FID_ATPaths_Buffers,distance_12_13_14_15_16_17_18_19,FID_ATPaths_Future_Buffers,distance_12_13_14_15_16_17_18_19_20,FID_ATCycleTracks_Buffers,distance_12_13_14_15_16_17_18_19_20_21,FID_ATCycleTracks_Future_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22,...,FID_ParksAndOpenSpace,FacilityID,FID_CommunityCenters_Buffers,distance_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28,FID_UtahQualifiedOpportunityZones,OBJECTID_12_13,QOZ,FID_WeberCommunities,FID_WeberCommunities_1,NAME,CommCode,FID_UintahHighlands,Shape_Length,Shape_Area,UNIQ_ID,CM,CU,CC,CN,AA,AT,TT,TF1,TF2,TF,TA1,TA2,TA3,TA4,TA,AC,AH,AG,AM,AE1,AE2,AE,AP,OZ,BC
0,1,1,29,63,2870,160040011,Governmental,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.002262,8.412697e-08,1,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,GO
1,2,1,29,72,11906,160120011,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003783,4.325147e-07,2,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NO
2,3,1,29,73,11907,160120012,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.007786,1.099263e-06,3,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NO
3,4,1,29,74,11912,160120029,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003640,3.852401e-07,4,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NO
4,5,1,29,75,11913,160120030,NaN,-1,NaN,-1,0,0,0.000000,0.000000,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,-1,0.00,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,...,-1,0,-1,0.0,-1,0,NaN,16,32197,Pleasant View,PV,-1,0.003219,3.821471e-07,5,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147064,147065,1,29,183231,27479,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,...,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147065,0,0,0,0,0.857134,0.391064,0.5,0.25,0.0,0.25,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,0.5,0.5,0.0,0.5,1,0,SF
147065,147066,1,29,183232,27480,NaN,Single Family Res,-1,NaN,258,307,570415,0.857134,0.391064,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.000000,1,0.25,3,2.0,-1,0.0,2,1.0,2,1.0,-1,0.0,2,1.0,...,1,0,2,1.0,-1,0,NaN,32,32193,South Ogden,SO,-1,0.000064,2.501400e-11,147066,0,0,0,0,0.857134,0.391064,0.5,0.25,0.0,0.25,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,0.5,0.5,0.0,0.5,1,0,SF
147066,147067,1,29,192694,1246,020300004,G

In [23]:
dfUnionedWeber_Scores = dfUnionedWeberWithBC[['UNIQ_ID','BC','OZ','CM','CU','CC','CN','AA','AT','TT','TF','TA','AC','AH','AE','AG','AP','AM']]
display(dfUnionedWeber_Scores)
display(dfUnionedWeber_Scores.describe())
display(dfUnionedWeber_Scores.sum())

,UNIQ_ID,BC,OZ,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM
0,1,GO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
1,2,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
2,3,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
3,4,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
4,5,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147064,147065,SF,0,0,0,0,0,0.857134,0.391064,0.5,0.25,0.5,0.0,1.0,0.5,0.0,1,0.5
147065,147066,SF,0,0,0,0,0,0.857134,0.391064,0.5,0.25,0.5,0.0,1.0,0.5,0.0,1,0.5
147066,147067,GO,1,0,0,0,0,0.778922,0.400341,1.0,0.00,1.0,1.0,1.0,0.5,1.0,1,0.5
147067,147068,SF,1,0,0,0,0,0.778922,0.400341,1.0,0.00,1.0,1.0,1.0,0.5,1.0,1,0.5


,UNIQ_ID,OZ,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM
count,147069.000000,147069.000000,147069.000000,147069.0,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000,147069.000000
mean,73535.000000,0.058850,0.017604,0.0,0.062984,0.015999,0.778985,0.340437,0.221369,0.100643,0.711085,0.436992,0.230201,0.519637,0.382742,0.357410,0.153516
std,42455.307707,0.235344,0.131507,0.0,0.242935,0.125473,0.201742,0.313872,0.302034,0.197909,0.373125,0.392319,0.347674,0.402651,0.400000,0.479239,0.296668
min,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,36768.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.737817,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,73535.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.824604,0.301041,0.000000,0.000000,1.000000,0.500000,0.000000,0.500000,0.500000,0.000000,0.000000
75%,110302.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.897427,0.561288,0.500000,0.250000,1.000000,1.000000,0.500000,1.000000,0.500000,1.000000,0.000000
max,147069.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


UNIQ_ID                                          10814718915
BC         GONONONONONONONOMFNONONONONONOAGAGAGAGAGAGAGAG...
OZ                                                      8655
CM                                                      2589
CU                                                         0
CC                                                      9263
CN                                                      2353
AA                                             114564.611943
AT                                              50067.741581
TT                                                   32556.5
TF                                                   14801.5
TA                                                  104578.5
AC                                                   64268.0
AH                                                   33855.5
AE                                                   76422.5
AG                                                   56289.5
AP                      

In [25]:
#export to csv
dfUnionedWeber_Scores.to_csv('..\\intermediate\\scores5.csv')

In [ ]:
# DONE MANUALLY SINCE PROGRAMATIC DIDN'T WORK
#arcpy.management.Copy(os.path.join(dirIntermediate,'scores5.csv'), "Scores5")

# CREATE INDEX IN SCORES BEFORE UNION TO MAKE FASTER

# union to parcels id only
arcpy.management.AddJoin("ParcelPartsIDsOnly", "OBJECTID", "Scores5", "UNIQ_ID")

arcpy.management.CopyFeatures("ParcelPartsIDsOnly", "ParcelPartsWithScores")

arcpy.management.RemoveJoin("ParcelPartsIDsOnly")

# Get Max Score per Category for Each Community

In [28]:
sdfPPwS = pd.DataFrame.spatial.from_featureclass("ParcelPartsWithScores")
sdfPPwS

,OBJECTID_1,CommCode,UNIQ_ID,OBJECTID,Field1,UNIQ_ID_1,BC,OZ,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM,SHAPE
0,1,PV,1,1,0,None,GO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,"{""rings"": [[[-111.9861708326178, 41.3545187634..."
1,2,PV,2,2,1,None,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,"{""rings"": [[[-112.00149455363072, 41.349620230..."
2,3,PV,3,3,2,None,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,"{""rings"": [[[-112.00284924020025, 41.348987901..."
3,4,PV,4,4,3,None,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,"{""rings"": [[[-111.99856759851582, 41.349467307..."
4,5,PV,5,5,4,None,NO,0,0,0,0,0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,"{""rings"": [[[-112.00022194818433, 41.349852756..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147064,147065,SO,147065,147065,147064,None,SF,0,0,0,0,0,0.857134,0.391064,0.5,0.25,0.5,0.0,1.0,0.5,0.0,1,0.5,"{""rings"": [[[-111.95832422658174, 41.167399674..."
147065,147066,SO,147066,147066,147065,None,SF,0,0,0,0,0,0.857134,0.391064,0.5,0.25,0.5,0.0,1.0,0.5,0.0,1,0.5,"{""rings"": [[[-111.95832422658174, 41.167399674..."
147066,147067,OG,147067,147067,147066,None,GO,1,0,0,0,0,0.778922,0.400341,1.0,0.00,1.0,1.0,1.0,0.5,1.0,1,0.5,"{""rings"": [[[-111.95416000128301, 41.223091858..."
147067,147068,OG,147068,147068,147067,None,SF,1,0,0,0,0,0.778922,0.400341,1.0,0.00,1.0,1.0,1.0,0.5,1.0,1,0.5,"{""rings"": [[[-111.95416000128301, 41.223091858..."


In [29]:
sdfPPwS.max()

OBJECTID_1    147069
CommCode          WW
UNIQ_ID       147069
OBJECTID      147069
Field1        147068
UNIQ_ID_1       None
BC                UT
OZ                 1
CM                 1
CU                 0
CC                 1
CN                 1
AA               1.0
AT               1.0
TT               1.0
TF               1.0
TA               1.0
AC               1.0
AH               1.0
AE               1.0
AG               1.0
AP                 1
AM               1.0
dtype: object

In [33]:
dfPPwS_MaxComm = sdfPPwS.groupby(['CommCode']).agg(CM=("CM",'max'),
                                                   CU=("CU",'max'),
                                                   CC=("CC",'max'),
                                                   CN=("CN",'max'),
                                                   AA=("AA",'max'),
                                                   AT=("AT",'max'),
                                                   TT=("TT",'max'),
                                                   TF=("TF",'max'),
                                                   TA=("TA",'max'),
                                                   AC=("AC",'max'),
                                                   AH=("AH",'max'),
                                                   AE=("AE",'max'),
                                                   AG=("AG",'max'),
                                                   AP=("AP",'max'),
                                                   AM=("AM",'max'))
dfPPwS_MaxComm

,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM
CommCode,,,,,,,,,,,,,,,
ED,0,0,0,1,1.000000,0.000000,0.5,0.00,1.0,1.0,0.0,1.0,1.0,0,0.0
FW,0,0,1,1,1.000000,1.000000,1.0,1.00,1.0,1.0,0.0,1.0,0.0,1,1.0
HA,0,0,1,1,1.000000,1.000000,0.5,0.25,1.0,1.0,0.5,1.0,1.0,1,1.0
HO,0,0,0,0,1.000000,0.152317,0.0,0.00,1.0,1.0,0.5,1.0,0.5,1,0.0
HU,0,0,0,1,0.286384,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,1.0
LI,0,0,0,0,0.989905,0.000000,0.0,0.00,1.0,0.5,0.0,0.5,0.5,0,0.0
MS,0,0,0,1,1.000000,1.000000,0.5,1.00,1.0,1.0,0.0,1.0,0.5,1,0.0
NO,0,0,1,1,1.000000,1.000000,0.5,0.00,1.0,1.0,1.0,1.0,1.0,1,1.0
OG,1,0,1,1,1.000000,1.000000,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1,1.0


In [34]:
dfPPwS_wValues = dfPPwS_MaxComm.loc[:, 'CM':'AM']/dfPPwS_MaxComm.loc[:, 'CM':'AM']
dfPPwS_wValues = dfPPwS_wValues.fillna(0)
dfPPwS_wValues

,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM
CommCode,,,,,,,,,,,,,,,
ED,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
FW,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
HA,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
HO,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
HU,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
LI,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
MS,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
NO,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
OG,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
dfPPwS_wValuesCol = dfPPwS_wValues.loc[:, 'CM':'AM'].replace(1, pd.Series(dfPPwS_MaxComm.columns, dfPPwS_MaxComm.columns))
dfPPwS_wValuesCol = dfPPwS_wValuesCol.loc[:, 'CM':'AM'].replace(0,"")
dfPPwS_wValuesCol

,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM
CommCode,,,,,,,,,,,,,,,
ED,,,,CN,AA,,TT,,TA,AC,,AE,AG,,
FW,,,CC,CN,AA,AT,TT,TF,TA,AC,,AE,,AP,AM
HA,,,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM
HO,,,,,AA,AT,,,TA,AC,AH,AE,AG,AP,
HU,,,,CN,AA,,,,,,,,,,AM
LI,,,,,AA,,,,TA,AC,,AE,AG,,
MS,,,,CN,AA,AT,TT,TF,TA,AC,,AE,AG,AP,
NO,,,CC,CN,AA,AT,TT,,TA,AC,AH,AE,AG,AP,AM
OG,CM,,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM


In [36]:
#concatenante column values into single field
cols = ['CM','CU','CC','CN','AA','AT','TT','TF','TA','AC','AH','AE','AG','AP','AM']
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol['combined'].str.replace(',,,,,',',')
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol['combined'].str.replace(',,,,' ,',')
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol['combined'].str.replace(',,,'  ,',')
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol['combined'].str.replace(',,'   ,',')
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol['combined'].str.lstrip(',')
dfPPwS_wValuesCol['combined'] = dfPPwS_wValuesCol['combined'].str.rstrip(',')
dfPPwS_wValuesCol['combined'] = '"' + dfPPwS_wValuesCol['combined'] + '"'
dfPPwS_wValuesCol['combined']

CommCode
ED                         "CN,AA,TT,TA,AC,AE,AG"
FW             "CC,CN,AA,AT,TT,TF,TA,AC,AE,AP,AM"
HA       "CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
HO                      "AA,AT,TA,AC,AH,AE,AG,AP"
HU                                     "CN,AA,AM"
LI                               "AA,TA,AC,AE,AG"
MS                "CN,AA,AT,TT,TF,TA,AC,AE,AG,AP"
NO          "CC,CN,AA,AT,TT,TA,AC,AH,AE,AG,AP,AM"
OG    "CM,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
OV          "CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
PC                      "CN,AA,TF,TA,AC,AE,AG,AP"
PV          "CC,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
RV          "CC,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
RY       "CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
SO       "CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
UH             "CC,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP"
UI             "AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
WC                         "CN,AA,TT,TA,AC,AE,AG"
WH             "CC,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP"
WT          "CC,AA,AT,TT,TF,TA,AC,AH,AE,A

In [ ]:
# manual changes
# REMOVE CC from FW (Farr West)